# Assignment 5: PCA and Neural Networks (50 marks)
### Due: April 4 at 11:59pm 

### Name: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Part 1: Principle Component Analysis (PCA) (20 marks)

### Step 1. Load data (2 marks)

You have been asked by an agricultural company to help them predict the type of wheat based on kernel characteristics. You have been given all the information you need in the seeds_dataset.txt file on D2L. The original dataset can be found [here](https://archive.ics.uci.edu/dataset/236/seeds)

The first step is to read in the file using pandas and inspect the first few columns. Note that the data is in a text file, so the separator is different compared to a csv file.

In [ ]:
# TO DO: Import dataset and inspect the first few columns (1 mark)


Next, you will need to separate the data into your feature matrix and target vector.

In [ ]:
# TO DO: Separate the data into feature matrix and target vector. Print the shape of X and y (1 mark)


### Steps 2+3: Preprocessing and Model Selection (3 marks)

The first step is to split the data into training and testing sets.

In [ ]:
# TO DO: Split the data into training and testing sets using 10% for test set (1 mark)


Now that we have our training data, we need to decide which preprocessing methods to use. Since we do not want any information leaking into the model validation stage, we will need to create a Pipeline. For this case, our model for the Pipeline is `Logistic Regression(max_iter=1000)`.

In [ ]:
# TO DO: Setup required preprocessing method(s) and model (1 mark)


In [ ]:
# TO DO: Setup pipeline (1 mark)


### Step 4: Validation (2 marks)

To validate the results, we need to use cross-validation. To make sure we are using the best hyperparmeters, we can use a grid search. The parameter grid has been provided below.

In [ ]:
param_grid = [{'classifier': [LogisticRegression(max_iter=1000)],
              'classifier__C': [0.01, 0.1, 1.0, 10.0, 100],
              'classifier__fit_intercept': [True, False]}]

In [ ]:
#TO DO: Implement the grid search and print the best parameters, training score, cross-validation score and test score (2 marks)


### Step 5: Visualize using PCA (5 marks)

Now you can visualize the results from Step 4 using PCA. Use the best parameters from the previous step to predict the label for the testing data. For the plot, each data point must be colored based on the class label.

In [ ]:
# TO DO: Predict target vector labels using best estimator (1 mark)


In [ ]:
# TO DO: Reduce dimensions of test data for plotting (2 marks)


In [ ]:
# TO DO: Plot test points colored by predicted label (2 marks)


### Questions (4 marks)

1. Did the logistic regression model work well for this dataset? Why would you make this conclusion? 
1. Did PCA work well for this model? How does this relate to the results from the model used? 

*ANSWER HERE*

### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

## Part 2: Neural Networks (28 marks)

For this assignment, you will be practicing using scikit-learn and TensorFlow to implement basic neural networks (MLP). The dataset we will be using is the energy dataset from Yellowbrick (https://www.scikit-yb.org/en/latest/api/datasets/energy.html).

### Step 1: Load data (1 mark)

You will need to load the file and split it into the feature matrix and target vector. Note that this dataset has two targets, heating load and cooling load. To retrieve the single target dataset, you need `return_dataset=False`. 

In [ ]:
# TO DO: Load dataset into feature matrix and target vector (1 mark)


### Step 2: Process your dataset (6 marks)

In [ ]:
# TO DO: Check if there are any missing values - if yes, decide how to fill them (1 mark)


In [ ]:
# TO DO: Check data type for each column and the number of unique values - do you need to encode any of them? (1 mark)


In [ ]:
# TO DO: Check the range of each feature - do you need to scale your data? (1 mark)


In [ ]:
# TO DO: Split your data into training and testing sets using 20% for test set (1 mark)


In [ ]:
# TO DO: Implement scaling and/or encoding here if needed (2 marks for preprocessing properly or justifying why it isn't needed)


### Step 3: Implement Neural Network (MLP) (8 marks)

### Part 1: Use scikit-learn (3 marks)

For each case, you will need to find the predicted target values.

In [ ]:
# TO DO: Test using default parameters and max_iter = 10000 (1 mark)
                

In [ ]:
# TO DO: Test using two hidden layers with 100 nodes each (1 mark)


In [ ]:
# TO DO: Test using three hidden layers with 100 nodes each (1 mark)


### Part 2: Use TensorFlow (5 marks)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

Instead of scaling the data using a scikit-learn scaler, you can scale the data using a normalization layer.

In [ ]:
# TO DO: Define normalization layer (1 mark)
 

Using `keras.Sequential`, implement an MLP with the same hidden layer setups as above:

In [ ]:
# TO DO: Implement MLP with one hidden layer with 100 nodes and the relu activation function (2 marks)
# Compile the model with loss='mean_absolute_error' and optimizer=tf.keras.optimizers.Adam(0.001)
# Fit the model using validation_split=0.2, verbose=0 and epochs=100


In [ ]:
# TO DO: Repeat with two hidden layers with 100 nodes each and the relu activation function (1 mark)


In [ ]:
# TO DO: Repeat with three hidden layers with 100 nodes each and the relu activation function (1 mark)


### Step 4: Compare the accuracy of both methods (4 marks)

For this part, calculate the mean absolute error for each model and print in a table using pandas

In [ ]:
# TO DO: Calculate the MAE for the three scikit-learn tests (1.5 marks)


In [ ]:
# TO DO: Calculate the MAE for the three TensorFlow tests (1.5 marks)


In [ ]:
# TO DO: Print the results (1 mark)


### Questions (5 marks)

1. Which model produced the least amount of error?
1. Do the results change if you run your code multiple times? Why or why not?
1.  Why are the numbers different between the scikit-learn and TensorFlow methods when we used the same number of hidden layers and hidden units per layer? List two potential reasons.

*ANSWER HERE*

### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

## Part 3: Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challangeing, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*